# dAiv AI_Competition[2024]_Basic Baseline for PyTorch

## Import Libraries

In [1]:
from os import path, mkdir, makedirs
import signal

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.cuda.amp import GradScaler

import torchvision
from torchvision import transforms

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

### Check GPU Availability

In [2]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [3]:
import os

# ** Please select CUDA DEVICES before training the model **
# Set CUDA_VISIBLE_DEVICES to specify which GPUs to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1, 4, 6"

# Set device to CUDA (which will use the GPUs specified above)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("INFO: Using device -", device)
print("Number of devices currently in use -", torch.cuda.device_count())

INFO: Using device - cpu
Number of devices currently in use - 0


## Load DataSets

In [4]:
from typing import Callable, Optional
from torchvision.datasets.utils import download_and_extract_archive

torchvision.datasets.utils.tqdm = tqdm


class FoodImageDataset(torchvision.datasets.ImageFolder):
    download_url = "https://daiv-cnu.duckdns.org/contest/ai_competition[2024]_basic/dataset/datasets.zip"

    def __init__(self, root: str, force_download: bool = True, train: bool = True, valid: bool = False, transform: Optional[Callable] = None, target_transform: 
                 Optional[Callable] = None):
        self.download(root, force=force_download)

        if train:
            if valid:
                root = path.join(root, "valid")
            else:
                root = path.join(root, "train")
        else:
            root = path.join(root, "test")

        super().__init__(root=root, transform=transform, target_transform=target_transform)

    @classmethod
    def download(cls, root: str, force: bool = False):
        if force or not path.isfile(path.join(root, "datasets.zip")):
            download_and_extract_archive(cls.download_url, download_root=root, extract_root=root, filename="datasets.zip")
            print("INFO: Dataset archive downloaded and extracted.")
        else:
            print("INFO: Dataset archive found in the root directory. Skipping download.")

In [5]:
# Image Resizing and Tensor Conversion
IMG_SIZE = (512, 512)
IMG_NORM = dict(  # ImageNet Normalization
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
)

resizer = transforms.Compose([
    transforms.Resize(IMG_SIZE),  # Resize Image
    transforms.ToTensor(),  # Convert Image to Tensor
    transforms.Normalize(**IMG_NORM)  # Normalization
])

In [8]:
DATA_ROOT = path.join(".", "data")

train_dataset = FoodImageDataset(root=DATA_ROOT, force_download=False, train=True, transform=resizer)
valid_dataset = FoodImageDataset(root=DATA_ROOT, force_download=False, valid=True, transform=resizer)
test_dataset = FoodImageDataset(root=DATA_ROOT, force_download=False, train=False, transform=resizer)

print(f"INFO: Dataset loaded successfully. Number of samples - Train({len(train_dataset)}), Valid({len(valid_dataset)}), Test({len(test_dataset)})")

  0%|          | 0/1163477903 [00:00<?, ?it/s]

Extracting ./data/datasets.zip to ./data
INFO: Dataset archive downloaded and extracted.
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Dataset loaded successfully. Number of samples - Train(9866), Valid(3430), Test(3347)


In [ ]:
import matplotlib.pyplot as plt
from collections import Counter

# 클래스별로 해당하는 이미지의 개수를 셉니다.
class_counts = Counter(train_dataset.targets)

# 클래스 인덱스를 클래스 이름으로 변환합니다.
class_names = train_dataset.classes  # ImageFolder의 클래스 이름들

# 클래스 인덱스를 클래스 이름으로 매핑합니다.
class_distribution = {class_names[i]: count for i, count in class_counts.items()}

# 분포를 시각화합니다.
plt.figure(figsize=(10, 6))
plt.bar(class_distribution.keys(), class_distribution.values(), color='skyblue')
plt.title('Class Distribution in Training Dataset')
plt.xlabel('Classes')
plt.ylabel('Number of Samples')
plt.xticks(rotation=45)
plt.tight_layout()

# 그래프를 표시합니다.
plt.show()

## Data Augmentation if needed

In [ ]:
ROTATE_ANGLE = 20
COLOR_TRANSFORM = 0.1

In [ ]:
augmenter = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(ROTATE_ANGLE),
    transforms.ColorJitter(
        brightness=COLOR_TRANSFORM, contrast=COLOR_TRANSFORM,
        saturation=COLOR_TRANSFORM, hue=COLOR_TRANSFORM
    ),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0), ratio=(0.75, 1.333)),
    resizer,
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value='random')
    
])

In [ ]:
train_dataset = FoodImageDataset(root=DATA_ROOT, force_download=False, train=True, transform=augmenter)

print(f"INFO: Train dataset has been overridden with augmented state. Number of samples - Train({len(train_dataset)})")

## DataLoader

In [ ]:
# Set Batch Size
if torch.cuda.device_count() > 2:
    BATCH_SIZE = 56
elif torch.cuda.device_count() == 2:
    BATCH_SIZE = 32
else:
    BATCH_SIZE = 32

print("INFO: BATCH_SIZE -", BATCH_SIZE)

In [ ]:
# CutMix 함수
def cutmix_data(x, y, alpha=1.0):
    # 랜덤으로 이미지 셔플
    indices = torch.randperm(x.size(0))
    shuffled_x = x[indices]
    shuffled_y = y[indices]

    # Beta 분포에서 lam 값을 생성하고 클립
    lam = np.clip(np.random.beta(alpha, alpha), 0.3, 0.7)

    # 랜덤 바운딩 박스 생성
    bbx1, bby1, bbx2, bby2 = rand_bbox(x.size(), lam)

    # 바운딩 박스 영역에 셔플된 이미지를 삽입
    x[:, :, bbx1:bbx2, bby1:bby2] = shuffled_x[:, :, bbx1:bbx2, bby1:bby2]

    # 실제로 혼합된 영역의 비율에 따라 lam 값을 재계산
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (x.size(-1) * x.size(-2)))

    return x, y, shuffled_y, lam

In [ ]:
# MixUp 함수
def mixup_data(x, y, alpha=1.0):
    lam = np.random.beta(alpha, alpha)

    # 셔플된 인덱스 생성
    indices = torch.randperm(x.size(0))
    shuffled_x = x[indices]
    shuffled_y = y[indices]

    # 이미지와 타깃 혼합
    mixed_x = lam * x + (1 - lam) * shuffled_x

    # 원본 타깃과 셔플된 타깃 반환
    y_a, y_b = y, shuffled_y

    return mixed_x, y_a, y_b, lam

# 임의의 바운딩 박스 생성 함수
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)

    # 랜덤 좌표 설정
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [ ]:
class CustomDatasetLoader(DataLoader):
    def __init__(self, *args, use_cutmix=False, use_mixup=False, alpha=1.0, **kwargs):
        super().__init__(*args, **kwargs)
        self.use_cutmix = use_cutmix
        self.use_mixup = use_mixup
        self.alpha = alpha

    def __iter__(self):
        for batch in super().__iter__():
            x, y = batch
            batch_size = x.size(0)

            # 비율에 따라 배치 크기 설정 (50%, 25%, 25%)
            half = batch_size // 2
            quarter = (batch_size - half) // 2
            remainder = batch_size - (half + quarter + quarter)

            x_transformed = x[:half]
            y_transformed = y[:half]

            x_mixup, y_a_mixup, y_b_mixup, lam_mixup = x[half:half+quarter], y[half:half+quarter], y[half:half+quarter], 1.0
            if self.use_mixup:
                x_mixup, y_a_mixup, y_b_mixup, lam_mixup = mixup_data(x_mixup, y_a_mixup, self.alpha)

            x_cutmix, y_a_cutmix, y_b_cutmix, lam_cutmix = x[half+quarter:], y[half+quarter:], y[half+quarter:], 1.0
            if self.use_cutmix:
                x_cutmix, y_a_cutmix, y_b_cutmix, lam_cutmix = cutmix_data(x_cutmix, y_a_cutmix, self.alpha)

            # 필요할 경우 잔여 이미지를 처리 (비율로 나누어 떨어지지 않는 경우)
            if remainder > 0:
                x_extra = x[-remainder:]
                y_extra = y[-remainder:]
                x_transformed = torch.cat((x_transformed, x_extra), dim=0)
                y_transformed = torch.cat((y_transformed, y_extra), dim=0)

            # 각 파트 결합
            x_final = torch.cat((x_transformed, x_mixup, x_cutmix), dim=0)
            y_final = torch.cat((y_transformed, y_a_mixup, y_a_cutmix), dim=0)  # MixUp과 CutMix의 첫 번째 라벨 사용
            y_final_b = torch.cat((y_transformed, y_b_mixup, y_b_cutmix), dim=0)

            # 최종 배치를 반환
            yield x_final, y_final, y_final_b, lam_mixup  # 여기서는 lam_mixup을 반환하지만 실제로는 필요에 따라 lam_cutmix도 함께 고려 가능

In [ ]:
# 기존 코드
MULTI_PROCESSING = True  # Set False if DataLoader is causing issues

from platform import system
if MULTI_PROCESSING and system() != "Windows":  # Multiprocess data loading is not supported on Windows
    import multiprocessing
    cpu_cores = multiprocessing.cpu_count() // 4
    # cpu_cores = 20
    print(f"INFO: Number of CPU cores - {cpu_cores}")
else:
    cpu_cores = 0
    print("INFO: Using DataLoader without multi-processing.")

In [ ]:
# CustomDatasetLoader 사용
train_loader = CustomDatasetLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=cpu_cores,
    use_cutmix=True,  # CutMix 사용 가능
    use_mixup=True,   # MixUp 사용 가능
    alpha=1.0
)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=cpu_cores)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=cpu_cores)

In [ ]:
# Image Visualizer
def imshow(image_list, mean=IMG_NORM['mean'], std=IMG_NORM['std']):
    np_image = np.array(image_list).transpose((1, 2, 0))
    de_norm_image = np_image * std + mean
    plt.figure(figsize=(10, 10))
    plt.imshow(de_norm_image)
    plt.axis('off')  # 축 제거
    plt.show()

In [ ]:
# 데이터 로더에서 이미지를 가져옴
data = next(iter(train_loader))

# CustomDatasetLoader의 설정에 따라 반환되는 값 처리
if len(data) == 2:
    images, targets = data
elif len(data) == 4:
    images, targets_a, targets_b, lam = data

In [ ]:
# 이미지 시각화
grid_images = torchvision.utils.make_grid(images, nrow=8, padding=20)
imshow(grid_images)

In [ ]:
class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Conv2d(in_channels, in_channels // reduction, kernel_size=1)
        self.relu = nn.ReLU()
        self.fc2 = nn.Conv2d(in_channels // reduction, in_channels, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x)
        y = self.fc1(y)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.sigmoid(y)
        return x * y.expand_as(x)

class ResNeXtBottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1, cardinality=32, base_width=4, reduction=16):
        super(ResNeXtBottleneck, self).__init__()
        width = int(planes * (base_width / 64.)) * cardinality
        # 1x1 Conv
        self.conv1 = nn.Conv2d(in_planes, width, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(width)
        # 3x3 Grouped Conv
        self.conv2 = nn.Conv2d(width, width, kernel_size=3, stride=stride, padding=1, groups=cardinality, bias=False)
        self.bn2 = nn.BatchNorm2d(width)
        # 1x1 Conv
        self.conv3 = nn.Conv2d(width, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)

        # SEBlock 추가
        self.se = SEBlock(planes * self.expansion, reduction)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * self.expansion)
            )

    def forward(self, x):
        out = nn.functional.relu(self.bn1(self.conv1(x)))
        out = nn.functional.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = self.se(out)  # SEBlock 적용
        out += self.shortcut(x)
        out = nn.functional.relu(out)
        return out


class ResNeXt(nn.Module):
    def __init__(self, block, num_blocks, num_classes=1000, cardinality=32, base_width=4):
        super(ResNeXt, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1, cardinality=cardinality, base_width=base_width)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2, cardinality=cardinality, base_width=base_width)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2, cardinality=cardinality, base_width=base_width)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2, cardinality=cardinality, base_width=base_width)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride, cardinality, base_width):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride, cardinality, base_width))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        x = nn.functional.relu(self.bn1(self.conv1(x)))
        x = nn.functional.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

def ResNeXt50_32x4d(num_classes=1000):
    return ResNeXt(ResNeXtBottleneck, [3, 4, 6, 3], num_classes=num_classes, cardinality=32, base_width=4)

def ResNeXt101_32x8d(num_classes=1000):
    return ResNeXt(ResNeXtBottleneck, [3, 4, 23, 3], num_classes=num_classes, cardinality=32, base_width=8)

## Define Model

In [ ]:
# Initialize Model
model = ResNeXt101_32x8d()
model_id = "ResNeXt101_32x8d"

# 다중 GPU 설정 (DataParallel 사용)
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

# 모델을 장치로 이동
model.to(device)

In [ ]:
LEARNING_RATE = 0.0003  # 학습률을 더 낮추어 더 안정적인 학습을 유도
WEIGHT_DECAY = 1e-5  # 모델이 더 크므로, Weight Decay를 조금 줄여 과적합 방지

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)  # AdamW 옵티마이저 사용
lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)  # 더 큰 T_max 값으로 학습률 조정 기간 연장

## Training Loop

In [ ]:
# 모델 저장 및 로드 함수 정의
def save_checkpoint(epoch, model, optimizer, loss, PATH):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }
    torch.save(checkpoint, PATH)
    print(f" Model saved.")

def load_checkpoint(PATH, model, optimizer):
    if path.isfile(PATH):
        checkpoint = torch.load(PATH)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        loss = checkpoint['loss']
        print(f"체크포인트 '{path.basename(PATH)}'에서 모델 로드 완료 (시작 에포크: {start_epoch})")
        return start_epoch, loss
    else:
        print(f"체크포인트 '{path.basename(PATH)}'를 찾을 수 없습니다. 새로 훈련을 시작합니다.")
        return 0, None

In [ ]:
from IPython.display import display
import ipywidgets as widgets

# Interactive Loss Plot Update
def create_plot():
    train_losses = []
    valid_losses = []

    # Enable Interactive Mode
    plt.ion()

    # Loss Plot Setting
    fig, ax = plt.subplots(figsize=(6, 4))
    train_line, = ax.plot(train_losses, label="Train Loss")
    valid_line, = ax.plot(valid_losses, label="Validation Loss")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss")
    ax.set_title("Cross Entropy Loss")
    ax.legend()

    # Display Plot
    plot = widgets.Output()
    display(plot)

    def update_plot(epoch, train_loss=None, valid_loss=None):
        if train_loss is not None:
            train_losses.append(train_loss)
            train_line.set_ydata(train_losses)
            train_line.set_xdata(range(1, len(train_losses) + 1))

        if valid_loss is not None:
            valid_losses.append(valid_loss)
            valid_line.set_ydata(valid_losses)
            valid_line.set_xdata(range(1, len(valid_losses) + 1))

        ax.relim()
        ax.autoscale_view()
        with plot:
            plot.clear_output(wait=True)
            display(fig)

    return update_plot

In [ ]:
def early_stopping(val_loss, best_val_loss, patience_counter, patience, model, epoch, optimizer, checkpoint_path):

    stop_training = False

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        save_checkpoint(epoch, model, optimizer, val_loss, checkpoint_path)  # 최상의 모델 저장
    else:
        patience_counter += 1

    if patience_counter >= patience:
        stop_training = True

    return best_val_loss, patience_counter, stop_training

In [ ]:
# Set Epoch Count
num_epochs = 200

In [ ]:
# Mixed Precision Training을 위한 GradScaler 생성
scaler = GradScaler()

# Early Stopping 설정
early_stopping_patience = 20  # 개선이 없을 때 기다릴 에폭 수
best_val_loss = np.inf
patience_counter = 0

# 디렉토리 생성
makedirs('checkpoints', exist_ok=True)
makedirs(path.join('checkpoints', 'early_stopping'), exist_ok=True)

PATH = path.join('checkpoints', f"{model_id}_checkpoint.pt.tar")
early_stopping_PATH = os.path.join('checkpoints', 'early_stopping', f"{model_id}_checkpoint.pt.tar")
save_cycle = 5

# 체크포인트 로드
start_epoch, _ = load_checkpoint(PATH, model, optimizer)

# 훈련 및 검증 루프
train_length, valid_length = map(len, (train_loader, valid_loader))

# 전체 진행 상황 표시를 위한 tqdm 설정
epochs = tqdm(range(start_epoch, num_epochs), desc="Epochs", position=0)
training_bar = tqdm(total=train_length, desc="Training", position=1, leave=True)
validation_bar = tqdm(total=valid_length, desc="Validation", position=2, leave=True)
update = create_plot()  # Create Loss Plot

for epoch in epochs:
    training_bar.reset(total=train_length)
    validation_bar.reset(total=valid_length)

    # Training
    model.train()
    running_train_loss = 0.0
    for i, batch in enumerate(train_loader):
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():  # AMP 사용
            if len(batch) == 4:
                inputs, targets_a, targets_b, lam = batch
                inputs, targets_a, targets_b = inputs.to(device), targets_a.to(device), targets_b.to(device)
                outputs = model(inputs)
                loss = lam * criterion(outputs, targets_a) + (1 - lam) * criterion(outputs, targets_b)
            else:
                inputs, targets = batch
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)

        scaler.scale(loss).backward()  # 스케일된 손실로 역전파 수행
        scaler.step(optimizer)  # 스케일러를 사용해 옵티마이저 업데이트
        scaler.update()  # 스케일러 업데이트

        running_train_loss += loss.item()
        training_bar.update(1)
        print(f"\rEpoch [{epoch+1}/{num_epochs}], Step [{i+1}/{train_length}], Loss: {loss.item():.6f}", end="")

    avg_train_loss = running_train_loss / train_length  # 평균 훈련 손실 계산

    val_acc, val_loss = 0, 0

    # Validation
    model.eval()
    with torch.no_grad():
        for inputs, targets in valid_loader:
            with torch.cuda.amp.autocast():  # AMP 사용
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)

                val_loss += criterion(outputs, targets).item() / valid_length
                val_acc += (torch.max(outputs, 1)[1] == targets.data).sum().item()

            validation_bar.update(1)

    val_acc /= len(valid_dataset)
    print(f"\rEpoch [{epoch+1:2}/{num_epochs}], Step [{train_length}/{train_length}], Loss: {loss.item():.6f}, Valid Acc: {val_acc:.6%}, Valid Loss: {val_loss:.6f}", end="\n" if (epoch+1) % save_cycle == 0 or (epoch+1) == num_epochs else "")

    # Update plots with average train and validation losses
    update(epoch + 1, train_loss=avg_train_loss, valid_loss=val_loss)

    # Early Stopping 체크
    best_val_loss, patience_counter, stop_training = early_stopping(
        val_loss, best_val_loss, patience_counter, early_stopping_patience, model, epoch, optimizer, early_stopping_PATH
    )

    if stop_training:
        print(f"Early stopping triggered at epoch {epoch+1}")
        break

    # save_cycle마다 모델 저장
    if (epoch + 1) % save_cycle == 0:
        save_checkpoint(epoch, model, optimizer, loss.item(), PATH)

# 마무리 출력
print("Training Completed!")

In [ ]:
if not path.isdir(path.join(".", "models")):
    mkdir(path.join(".", "models"))

# Model Save
save_path = path.join(".", "models", f"{model_id}.pt")
torch.save(model.state_dict(), save_path)
print(f"Model saved to {save_path}")

# Model Evaluation

In [ ]:
# Load Model
model = ResNeXt101_32x8d()

if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model)
model.load_state_dict(torch.load(path.join(".", "models", f"{model_id}.pt")))
model.to(device)

In [ ]:
results = dict(id=[], label=[])
test_length = len(test_dataset)

model.eval()
with torch.no_grad():
    for inputs, ids in tqdm(test_loader):
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        results['id'] += [test_dataset.classes[i] for i in ids]
        results['label'] += [train_dataset.classes[i] for i in preds.cpu().detach().numpy().tolist()]

In [ ]:
# Save Results
results_df = pd.DataFrame(results)

submission_dir = "submissions"
if not path.isdir(submission_dir):
    mkdir(submission_dir)

submit_file_path = path.join(submission_dir, f"{model_id}.csv")
results_df.to_csv(submit_file_path, index=False)
print("File saved to", submit_file_path)

results_df.head()

In [ ]:
# 학습 종료 시 GPU 메모리 해제
torch.cuda.empty_cache()

# 현재 프로세스 PID를 가져와 종료
pid = os.getpid()
os.kill(pid, signal.SIGKILL)